# Коллаборативная фильтрация (Collaborative Filtering)


---


* Для рекомендации используется история оценок как самого пользователя, так и других пользователей.
* Более универсальный подход, часто дает лучший результат.
* Есть свои проблемы (например, холодный старт).

В большинстве случаев алгоритмы коллаборативной фильтрации (CF) показывают лучший результат, чем content-based системы.


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd


data = {
    'userId': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
               2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
               3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
               4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
               5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
               6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
               7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
               8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
               9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
               10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
    'itemId': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] * 10,
    'rating': [0, 4, 0, 2, 1, 0, 5, 4, 0, 2,   # userId 1
               0, 5, 4, 3, 2, 1, 0, 5, 4, 3,   # userId 2
               2, 1, 0, 5, 4, 3, 2, 1, 0, 5,   # userId 3
               4, 3, 2, 1, 0, 5, 4, 3, 2, 1,   # userId 4
               0, 5, 4, 3, 2, 1, 0, 5, 4, 3,   # userId 5
               2, 1, 0, 5, 4, 3, 2, 1, 0, 5,   # userId 6
               4, 3, 2, 1, 0, 5, 4, 3, 2, 1,   # userId 7
               0, 5, 4, 3, 2, 1, 0, 5, 4, 3,   # userId 8
               2, 1, 0, 5, 4, 3, 2, 1, 0, 5,   # userId 9
               4, 3, 2, 1, 0, 5, 4, 3, 2, 1]   # userId 10
}

df = pd.DataFrame(data)

n_users = df['userId'].unique().shape[0]
n_items = df['itemId'].unique().shape[0]

input_list = df['itemId'].unique()


def scale_item_id(input_id):
    return np.where(input_list == input_id)[0][0] + 1

df['itemId'] = df['itemId'].apply(scale_item_id)


train_data, test_data = train_test_split(df, test_size=0.20)

# Создаем две user-item матрицы – для обучения и для теста
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

# Вычисляем сходство пользователей
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

# Функция предсказания
def predict(ratings, similarity, type):
    if type == 'user':
      mean_user_rating = ratings.mean(axis=1)
      ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
      pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
      mean_item_rating = ratings.mean(axis=0)
      ratings_diff = (ratings - mean_item_rating[np.newaxis, :])
      pred = mean_item_rating[np.newaxis, :] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=0)]).T
    return pred

# Получаем предсказания
user_prediction = predict(train_data_matrix, user_similarity, 'item')

# Для оценки качества предсказания (RMSE)
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

# Функция для получения рекомендаций
def get_recommendations(user_id, user_prediction, original_df, n_recommendations):
    user_idx = user_id - 1
    user_ratings = user_prediction[user_idx]

    # Оцененные песни
    rated_movies = original_df[(original_df['userId'] == user_id) & ((original_df['rating'] > 0)) ]['itemId'].values

    # Исключаем прослушенные
    recommendations = []
    for movie_id, predicted_rating in enumerate(user_ratings, start=1):
        if movie_id not in rated_movies:
            recommendations.append((movie_id, predicted_rating))

    # Сортируем рекомендации в порядке убывания
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Выбираем первые n_recommendations песен
    return recommendations[:n_recommendations]

# Получаем рекомендации для пользователя
print(get_recommendations(1, user_prediction, df, 5))
df

[(6, 2.5766983359405455), (1, 1.7015421914461624), (9, 1.633515845623838), (3, 1.2207320252969902)]


,userId,itemId,rating
0,1,1,0
1,1,2,4
2,1,3,0
3,1,4,2
4,1,5,1
...,...,...,...
95,10,6,5
96,10,7,4
97,10,8,3
98,10,9,2


# **User-based алгоритм**

---

Заменим жесткую кластеризацию на предположение, что объект понравится пользователю, если он понравился похожим пользователям. Тогда предпочтение пользователя u к объекту i можно записать следующим образом:

$$r_{ui} = \bar{r}_u + \frac{\sum_{v \in U_i} sim(u, v)(r_{vi} - \bar{r}_v)}{\sum_{v \in U_i} sim(u, v)}$$


* $\bar{r}_u$ — средняя оценка, проставленная пользователем $u$
* $sim(u,v)$ — мера схожести пользователей $u$ и $v$. В нашем случае это косинусная мера.

Косинусная мера:
$$similarity = \frac{A \cdot B}{\|A\|\|B\|} = \frac{\sum_{i=1}^{n} A_i \times B_i}{\sqrt{\sum_{i=1}^{n} (A_i)^2} \times \sqrt{\sum_{i=1}^{n} (B_i)^2}}$$


Однако у этого алгоритма есть недостатки:

* холодный старт — новые объекты никому не рекомендуются
* нечего рекомендовать новым/нетипичным пользователям

# **Item-based алгоритм**
---
Также имеется абсолютно симметричный алгоритм. Теперь будем считать, что объект понравится пользователю, если ему понравились похожие объекты. Предпочтение пользователя u к объекту i запишется так:

$$r_{ui} = \bar{r}_i + \frac{\sum_{j \in I_u} sim(i, j)(r_{uj} - \bar{r}_j)}{\sum_{j \in I_u} sim(i, j)}$$

* $\bar{r}_i$— средняя оценка, проставленная объекту $i$
* $sim(i,j)$ — мера схожести объектов $i$ и $j$.

У такого подхода остается недостаток в виде холодного старта и при этом рекомендации становятся тривиальными.

Cтоит отметить, что ресурсоемкость вычислений такими методами высока: для предсказаний необходимо держать в памяти все оценки всех пользователей.


#**Измерение качества рекомендаций**
---
Зачастую качество рекомендаций измеряется с помощью функции ошибки RMSE(Root Mean Squared Error):

$$RMSE = \sqrt{\frac{1}{|D|} \sum_{(u,i) \in D} (r_{ui} - \hat{r}_{ui})^2}$$

Данный способ, хоть и является стандартным для измерением качества, имеет ряд недостатков:



*   пользователи с большим разбросом оценок будут влиять на значение метрики больше, чем остальные
*   ошибка в предсказании высокой оценки имеет такой же вес, что и ошибка в предсказании низкой оценки
*   есть риск плохого ранжирования при почти идеальной RMSE и наоборот

Существуют при этом и другие метрики — метрики ранжирования, на основе полноты и точности. Однако они не так популярны и используются значительно реже.